## 1. Load libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

## 2. Load Data

In [3]:
files = {
    "business": "/content/drive/MyDrive/DAB322_Capstone 1_Group 9/Yelp_Datasets/Original_JSON_files/yelp_academic_dataset_business.json",
    "checkin": "/content/drive/MyDrive/DAB322_Capstone 1_Group 9/Yelp_Datasets/Original_JSON_files/yelp_academic_dataset_checkin.json",
    "photos": "/content/drive/MyDrive/DAB322_Capstone 1_Group 9/Yelp_Datasets/Original_JSON_files/photos.json",
    "review": "/content/drive/MyDrive/DAB322_Capstone 1_Group 9/Yelp_Datasets/Original_JSON_files/yelp_academic_dataset_review.json",
    "tip": "/content/drive/MyDrive/DAB322_Capstone 1_Group 9/Yelp_Datasets/Original_JSON_files/yelp_academic_dataset_tip.json",
    "user": "/content/drive/MyDrive/DAB322_Capstone 1_Group 9/Yelp_Datasets/Original_JSON_files/yelp_academic_dataset_user.json"
}

# limit the number of rows
limit = 80000

datasets = {}
for name, path in files.items():
    df = pd.read_json(path, lines=True, nrows=limit)
    datasets[name] = df
    print(f"{name}: {len(df)} rows read")

# files name
business = datasets["business"]
checkin = datasets["checkin"]
photos = datasets["photos"]
review = datasets["review"]
tip = datasets["tip"]
user = datasets["user"]

business: 80000 rows read
checkin: 80000 rows read
photos: 80000 rows read
review: 80000 rows read
tip: 80000 rows read
user: 80000 rows read


## 4. Data Cleaning

### 4.1. Business

In [10]:
# Handle missing values in optional business fields
business['categories'] = business['categories'].fillna('Unknown')
business['address'] = business['address'].fillna('Unknown')
business['postal_code'] = business['postal_code'].fillna('Unknown')
business['attributes'] = business['attributes'].fillna('Unknown')
business['hours'] = business['hours'].fillna('Unknown')

# Standardize text formatting
business['city'] = business['city'].str.title()
business['state'] = business['state'].str.upper()

# Validate star ratings (must be between 0 and 5)
invalid_stars = business[(business['stars'] > 5) | (business['stars'] < 0)]
print(f"Invalid business star ratings:\n{invalid_stars}")

# Extract primary category from category list
business['main_cat'] = business['categories'].apply(
    lambda x: str(x).split(',')[0].strip() if pd.notnull(x) else 'Unknown'
)

# Normalize categories into high-level groups
def map_business_category(category):
    category = str(category).lower()

    if any(k in category for k in ['restaurant', 'food', 'cafe', 'pizza', 'coffee']):
        return 'Food & Beverage'
    if any(k in category for k in ['health', 'clinic', 'fitness', 'spa']):
        return 'Health & Wellness'
    if any(k in category for k in ['hotel', 'inn', 'bnb']):
        return 'Accommodation'
    if any(k in category for k in ['bar', 'nightlife', 'pub']):
        return 'Nightlife'
    if any(k in category for k in ['shop', 'store', 'retail']):
        return 'Retail'
    if any(k in category for k in ['auto', 'car', 'mechanic']):
        return 'Automotive'
    return 'Other'

business['main_cat'] = business['main_cat'].apply(map_business_category)

# Save cleaned business data
business.to_csv('business_cleaned.csv', index=False)


Invalid business star ratings:
Empty DataFrame
Columns: [business_id, name, address, city, state, postal_code, latitude, longitude, stars, review_count, is_open, attributes, categories, hours]
Index: []


### 4.2. Photos

In [11]:
# Fill missing optional text fields
photos['caption'] = photos['caption'].fillna('No caption')
photos['label'] = photos['label'].fillna('Unknown')

# Ensure identifier fields are stored as strings
photos['photo_id'] = photos['photo_id'].astype(str)
photos['business_id'] = photos['business_id'].astype(str)

# Remove duplicate photo records
photos = photos.drop_duplicates(subset='photo_id')

# Save cleaned photos dataset
photos.to_csv('photos_cleaned.csv', index=False)

print("Photos dataset cleaned and saved.")


Photos dataset cleaned and saved.


### 4.3. Review

In [12]:
# Keep only relevant columns
review = review[['review_id', 'user_id', 'business_id', 'stars', 'text', 'date']]

# Fill missing values (text + stars)
review['text'] = review['text'].fillna('No review text')
review['stars'] = review['stars'].fillna(0)

# Convert date to datetime
review['date'] = pd.to_datetime(review['date'], errors='coerce')

# Ensure ID columns are strings
review['review_id'] = review['review_id'].astype(str)
review['user_id'] = review['user_id'].astype(str)
review['business_id'] = review['business_id'].astype(str)

# Remove duplicate reviews
review = review.drop_duplicates(subset='review_id')

# Save cleaned review dataset
review.to_csv('review_cleaned.csv', index=False)


### 4.4. Checkin

In [13]:
# Split comma-separated check-in timestamps into a list
checkin['date_list'] = checkin['date'].str.split(',')

# Expand list into multiple rows (one row per check-in)
checkin = checkin.explode('date_list')

# Convert to datetime
checkin['checkin_time'] = pd.to_datetime(checkin['date_list'].str.strip(), errors='coerce')

# Drop invalid check-in timestamps
checkin = checkin.dropna(subset=['checkin_time'])

# Extract time features
checkin['year'] = checkin['checkin_time'].dt.year
checkin['month'] = checkin['checkin_time'].dt.month
checkin['day_of_week'] = checkin['checkin_time'].dt.dayofweek
checkin['hour'] = checkin['checkin_time'].dt.hour

# Drop raw columns that are no longer needed
checkin = checkin.drop(columns=['date', 'date_list'])

# Save cleaned check-in dataset
checkin.to_csv('checkin_cleaned.csv', index=False)

### 4.5. User

In [14]:
# Drop high-cardinality column (optional)
user_clean = user.drop(columns=['friends'], errors='ignore')

# Fill missing name and elite fields
user_clean['name'] = user_clean['name'].fillna('Unknown')
user_clean['elite'] = user_clean['elite'].fillna('None')

# Convert yelping_since to datetime
user_clean['yelping_since'] = pd.to_datetime(user_clean['yelping_since'], errors='coerce')

# Remove duplicate users
user_clean = user_clean.drop_duplicates(subset='user_id')

# Save cleaned user dataset
user_clean.to_csv('user_cleaned.csv', index=False)

### 4.6. Tip

In [15]:
# Remove empty tip text
tip = tip[tip['text'].str.strip().notna()]

# Convert date to datetime
tip['date'] = pd.to_datetime(tip['date'], errors='coerce')

# Drop rows missing key identifiers
tip = tip.dropna(subset=['user_id', 'business_id'])

# Remove duplicate tips
tip = tip.drop_duplicates(subset=['user_id', 'business_id', 'text'])

# Save cleaned tip dataset
tip.to_csv('tip_cleaned.csv', index=False)